In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import math
from torch import nn

In [2]:
X = torch.randn(1, 2, 4)  # bs, len, dim
X.shape

torch.Size([1, 2, 4])

In [3]:
d_model = 4
n_head = 2

In [6]:
class multi_head_attn(nn.Module):
    
    def __init__(self, d_model, n_head) -> None:
        super(multi_head_attn, self).__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_out = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, q, k, v):
        bs, lenth, dim = q.shape
        n_d = self.d_model // self.n_head
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)
        print(q.shape)
        
        q = q.view(bs, lenth, self.n_head, n_d).permute(0,2,1,3)
        print(q.shape)
        k = k.view(bs, lenth, self.n_head, n_d).permute(0,2,1,3)
        v = v.view(bs, lenth, self.n_head, n_d).permute(0,2,1,3)
        
        score = q @ k.transpose(-2, -1) / math.sqrt(n_d)
        print(score)
        mask = torch.tril(torch.ones(lenth, lenth))
        print(mask)
        score = score.masked_fill(mask == 0, float("-inf"))
        print(score)
        score = self.softmax(score) @ v
        score = score.permute(0, 2, 1, 3).contiguous().view(bs, lenth, -1)
        
        output = self.w_out(score)
        return output

In [7]:
attention = multi_head_attn(d_model, n_head)
output = attention(X, X, X)
print(output, output.shape)

torch.Size([1, 2, 4])
torch.Size([1, 2, 2, 2])
tensor([[[[ 0.3126,  0.1033],
          [ 0.4538,  0.0114]],

         [[-0.3053, -0.0160],
          [-0.1761,  0.0299]]]], grad_fn=<DivBackward0>)
tensor([[1., 0.],
        [1., 1.]])
tensor([[[[ 0.3126,    -inf],
          [ 0.4538,  0.0114]],

         [[-0.3053,    -inf],
          [-0.1761,  0.0299]]]], grad_fn=<MaskedFillBackward0>)
tensor([[[-0.4721,  0.0959, -0.4633,  0.1413],
         [-0.4681,  0.0861, -0.4088,  0.1041]]], grad_fn=<ViewBackward0>) torch.Size([1, 2, 4])
